In [ ]:
import pandas as pd
import geopandas as gpd
import unicodedata,time,os,csv
from datetime import datetime
import requests
import os, glob
import io
from zipfile import ZipFile

## Partie 1 : préparation des données communales

Téléchargement des aires urbaines

In [ ]:
url =  r"https://www.insee.fr/fr/statistiques/fichier/4803954/AAV2020_au_01-01-2022.zip"

r = requests.get(url, allow_redirects=True)
open('AAV2020_au_01-01-2022.zip', 'wb').write(r.content)

def read_zip(zip_fn, extract_fn=None):
    zf = ZipFile(zip_fn)
    if extract_fn:
        return zf.read(extract_fn)
    else:
        return {name:zf.read(name) for name in zf.namelist()}
    
df_ZAU = pd.read_excel(io.BytesIO(read_zip(r'AAV2020_au_01-01-2022.zip', 'AAV2020_au_01-01-2022.xlsx')),sheet_name=1,skiprows=5)

df_ZAU.head()

Récupération dans  de la couche des communes

In [ ]:
# url =  r"https://www.insee.fr/fr/statistiques/fichier/4803954/fonds_aav2020_2022.zip"
df_com = gpd.GeoDataFrame.from_file("./data/communes_2022/COMMUNE.shp")
df_com.rename(columns={'INSEE_COM':'CODGEO'},inplace=True)

In [ ]:
df_com = pd.merge(df_com,df_ZAU[["CODGEO","CATEAAV2020",'AAV2020']],left_on='CODGEO', right_on='CODGEO')

In [ ]:
#groupby des ZAU avec somme de la population
df_ZAUgpy = df_com.groupby(['AAV2020'])['POPULATION'].sum().reset_index()

# Jointure du groupby avec les communes
df_com = pd.merge(df_com,df_ZAUgpy,left_on='AAV2020', right_on='AAV2020')

# On renomme les colonnes
df_com.rename(columns={"POPULATION_x": "population", "POPULATION_y": "poppuzau"},inplace=True)

# On supprime les colonnes inutiles
df_com.drop(columns=["INSEE_CAN",'INSEE_ARR','ID','SIREN_EPCI'],inplace= True)

In [ ]:
df_com = df_com.to_crs("EPSG:3857")

In [ ]:
df_com.to_file("./data/communes_2022/com_zau.gpkg", layer='com_zau', driver="GPKG")

## Partie 2 : intersect entre CLC et com_zau. A faire sous Qgis

## Partie 3 : Définition des environnemes. Basé sur la méthodologie développée par M.Hillal

In [11]:
df_com_zau_clc = gpd.GeoDataFrame.from_file("./data/com_zau_clc.gpkg")

In [12]:
df_com_zau_clc.head()

,NOM,CODGEO,population,INSEE_DEP,INSEE_REG,CATEAAV2020,poppuzau,code_18,geometry
0,Connangles,43076,137,43,84,30,4449611,242,"MULTIPOLYGON (((408796.779 5669870.250, 408796..."
1,Connangles,43076,137,43,84,30,4449611,231,"MULTIPOLYGON (((404654.639 5665165.973, 404612..."
2,Connangles,43076,137,43,84,30,4449611,231,"MULTIPOLYGON (((406451.668 5671687.901, 406453..."
3,Connangles,43076,137,43,84,30,4449611,231,"MULTIPOLYGON (((406241.949 5668539.368, 406097..."
4,Connangles,43076,137,43,84,30,4449611,242,"MULTIPOLYGON (((402029.374 5665293.460, 401861..."


In [14]:
df_com_zau_clc.rename(columns={'code_18':'CODE_CLC', 'population':'POPULATION', 'poppuzau':'POPPUZAU'},inplace = True)
df_com_zau_clc = df_com_zau_clc.astype({"CATEAAV2020": str, "CODE_CLC": str})

In [18]:
test = df_com_zau_clc.head(100000)

In [33]:
def getPERIM(df):
	if (df["POPULATION"] > 50000 and (df["CATEAAV2020"] == '11' or df["CATEAAV2020"] == '12' or df["CATEAAV2020"] == '13') and df["POPPUZAU"] > 200000 and (df["CODE_CLC"].startswith('1') or df["CODE_CLC"] == '511')) :
		ENV_GEO = "111MH"
	elif (df["POPULATION"] > 50000 and (df["CATEAAV2020"] == '11' or df["CATEAAV2020"] == '12' or df["CATEAAV2020"] == '13') and  100000 <= df["POPPUZAU"] <= 200000  and (df["CODE_CLC"].startswith('1') or df["CODE_CLC"] == '511')) :
		ENV_GEO = "112MH"
	elif (df["POPULATION"] > 50000 and (df["CATEAAV2020"] == '11' or df["CATEAAV2020"] == '12' or df["CATEAAV2020"] == '13') and df["POPPUZAU"] < 100000  and (df["CODE_CLC"].startswith('1') or df["CODE_CLC"] == '511') ) :
		ENV_GEO = "113MH"
	elif (df["POPULATION"] < 50000 and (df["CATEAAV2020"] == '11' or df["CATEAAV2020"] == '12' or df["CATEAAV2020"] == '13' or df["CATEAAV2020"] == '20') and df["POPPUZAU"] > 200000 and (df["CODE_CLC"].startswith('1') or df["CODE_CLC"] == '511')) :
		ENV_GEO = "121MH"
	elif (df["POPULATION"] < 50000 and (df["CATEAAV2020"] == '11' or df["CATEAAV2020"] == '12' or df["CATEAAV2020"] == '13' or df["CATEAAV2020"] == '20') and  100000 <= df["POPPUZAU"] <= 200000 and (df["CODE_CLC"].startswith('1') or df["CODE_CLC"] == '511')) :
		ENV_GEO = "122MH"	
	elif (df["POPULATION"] < 50000 and (df["CATEAAV2020"] == '11' or df["CATEAAV2020"] == '12' or df["CATEAAV2020"] == '13' or df["CATEAAV2020"] == '20') and  df["POPPUZAU"] < 100000 and (df["CODE_CLC"].startswith('1') or df["CODE_CLC"] == '511')) :
		ENV_GEO = "123MH"	
	elif ((df["CATEAAV2020"] == '20' or df["CATEAAV2020"] == '30') and (df["CODE_CLC"].startswith('1') or df["CODE_CLC"] == '511')) :
		ENV_GEO = "124MH"
	elif not (df["CODE_CLC"].startswith('1') or df["CODE_CLC"] == '511')  :
		ENV_GEO = "130MH"	
	else :
		ENV_GEO = 'PB' 
	return ENV_GEO

df_com_zau_clc['env_geo'] = df_com_zau_clc.apply(getPERIM, axis=1)

In [34]:
df_com_zau_clc.env_geo.unique()

array(['130MH', '124MH', '123MH', '121MH', '111MH', '122MH', '112MH',
       '113MH'], dtype=object)

In [30]:
df_com_zau_clc.to_file("./data/test.gpkg", layer='test', driver="GPKG")